# Reading in MCTS (Monte Carlo Tree Search) data
### The goal is to train two RNNs.
 - Predict Probability of each of the moves being successful
 - Evaluate the value of the current board position

In [2]:
import pandas as pd

df = pd.read_pickle('Data/mcts_rand_trial_data.pkl')
cols = ['Board', 'Moves','Move_Probs','Reward', 'Weights']

print(df.head)
print(df.ndim, ' dimensions')
print(df.shape, ' shape')
print(df.size, ' size')

<bound method NDFrame.head of                                                 Board  \
0   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
1   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
2   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
3   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
4   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
..                                                ...   
45  [[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0], [...   
46  [[1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0], [...   
47  [[1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0], [...   
48  [[1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0], [...   
49  [[1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0], [...   

                                                Moves  \
0   [[4, 4, 7, 5, 0, 0], [3, 4, 16, 3, 0, 1], [4, ...   
1   [[8, 9, 20, 3, 0, 1], [10, 8, 9, 3, 0, 2], [10...   
2   [[4, 1, 9, 1, 4, 2], [4, 1, 11, 1, 0, 1], [2, ...   
3   [[10, 11, 19, 3, 3, 0], [9, 11, 14, 5, 4, 3], ...   


In [4]:
from board import Board
  
# checking out to make the data is stored correctly
  
row = 3098
print(df.iloc[row])
x = df.iloc[row]['Board']
board = Board(14)
board.show_dots = False
board.board = x
print(board)
print(df.iloc[row]['Move_Probs'])
print(df.iloc[row]['Moves'])


Board         [[0, 0, 0, 0, 0, 0, 0, -1, 0, -1, -1, -1, 0, 0...
Moves         [[11, 12, 11, 3, 5, 0], [10, 13, 7, 1, 5, 0], ...
Move_Probs    [0.00390625, 0.00390625, 0.00390625, 0.0039062...
Reward                                                       -1
Weights       [47.68156278379998, 26.012863876978756, 56.365...
Num_Sims                                                    363
mc_type       <bound method Board.monte_carlo_turn of <board...
Name: 19, dtype: object
   A B C D E F G H I J K L M N
 1 □ □ □ □ □ □ □ ▣ □ ▣ ▣ ▣ □ □ 
 2 □ □ □ □ □ ▣ ▣ ▣ □ ▣ □ □ ▣ □ 
 3 □ □ ▣ ▣ □ □ □ ▣ □ ▣ □ ▣ ▣ □ 
 4 □ ▣ ▣ □ □ □ ▣ □ ▣ □ □ ▣ □ □ 
 5 □ □ □ ▣ ▣ ▣ ▣ ▣ □ ▣ ▣ □ ▣ □ 
 6 □ □ □ ▣ □ □ □ □ □ ▣ ▣ ▣ ▣ ▣ 
 7 □ □ ▣ ▣ □ ▣ ▣ ▣ ▣ □ ▣ □ ▣ □ 
 8 ▣ ▣ □ □ ▣ □ □ □ □ □ ▣ □ □ □ 
 9 □ ▣ □ □ ▣ ▣ □ □ □ ▣ ▣ □ □ □ 
10 □ ▣ □ □ ▣ ▣ □ □ □ ▣ □ ▣ ▣ □ 
11 □ □ □ □ □ □ ▣ ▣ □ ▣ □ ▣ ▣ ▣ 
12 ▣ □ □ □ □ ▣ □ ▣ ▣ □ □ □ □ □ 
13 ▣ ▣ ▣ □ □ ▣ □ □ ▣ □ □ □ □ □ 
14 ▣ □ □ ▣ ▣ ▣ □ □ □ □ □ □ □ □ 

[np.float64(0.00390625), np.float64(0.00390625),

# Working on the DCNNs
### References:
 - https://medium.com/aiguys/deep-convolutional-neural-networks-dcnns-explained-in-layman-terms-b990b2818061
  - https://nikcheerla.github.io/deeplearningschool/2018/01/01/AlphaZero-Explained/


In [5]:
import tensorflow as tf

df = df.sample(frac = 1)
training_ratio = 0.8
train_df = df.iloc[:training_ratio(df.shape[0])]
test_df = df.iloc[training_ratio(df.shape[0]):]

train_boards = train_df['Boards']
train_rewards = train_df['Rewards']

test_boards = test_df['Boards']
test_rewards = test_df['Rewards']

model = tf.keras.models.Sequential([
    
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
  
])

# Compiling the model, Optimizer chosen is 'adam'
# Loss function chosen is 'sparse_categorical_crossentropy`
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training/fitting the model for 10 epochs
model.fit(train_boards, train_rewards, epochs=10)

test_loss = model.evaluate(test_boards, test_rewards)

print("\nTest accuracy: ",test_loss[1])



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1077, in launch

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1077, in launch

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1077, in launch

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import